In [2]:
import torch
import torch.nn.functional as F
if torch.cuda.is_available():
    torch.device("cuda")

In [10]:
rgb = []
colors = []
categories = []
for data in open("labels.csv", "r").read().splitlines()[1:]:
    r, g, b, label = data.split(",")
    rgb.append((float(r)/255, float(g)/255, float(b)/255))
    colors.append(label)

categories = sorted(list(set(colors)))
colors_enc = [categories.index(c) for c in colors]
print(categories, colors[0])

['Black', 'Blue', 'Brown', 'Green', 'Grey', 'Orange', 'Pink', 'Purple', 'Red', 'White', 'Yellow'] Blue


In [31]:
inputs = torch.tensor(rgb).float()
labels = F.one_hot(torch.tensor(colors_enc)).float()

hidden_layer_dim = 50

W1 = torch.randn((3, hidden_layer_dim)) * (5/3)/(3**0.5)
W2 = torch.randn((hidden_layer_dim, labels.shape[1])) * 0.01
print(inputs.shape, labels.shape)

# BatchNorm parameters
bngain = torch.ones((1, hidden_layer_dim))
bnbias = torch.zeros((1, hidden_layer_dim))
bnmean_running = torch.zeros((1, hidden_layer_dim))
bnstd_running = torch.ones((1, hidden_layer_dim))

parameters = [W1, W2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

torch.Size([5052, 3]) torch.Size([5052, 11])


In [32]:
for i in range(10000):
    hpreact = inputs @ W1
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
    h = torch.tanh(hpreact) # hidden layer
    logits = torch.tanh(h @ W2)
    loss = F.cross_entropy(logits.softmax(dim=1), labels)

    if i % 1000 == 0:
        print(loss)
    
    for p in parameters:
        p.grad = None
    loss.backward()
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    
    # break

tensor(2.3981, grad_fn=<DivBackward1>)
tensor(2.2538, grad_fn=<DivBackward1>)
tensor(2.2360, grad_fn=<DivBackward1>)
tensor(2.2248, grad_fn=<DivBackward1>)
tensor(2.2140, grad_fn=<DivBackward1>)
tensor(2.2026, grad_fn=<DivBackward1>)
tensor(2.1972, grad_fn=<DivBackward1>)
tensor(2.1933, grad_fn=<DivBackward1>)
tensor(2.1897, grad_fn=<DivBackward1>)
tensor(2.1859, grad_fn=<DivBackward1>)


In [24]:
color = torch.tensor([255,0,0]).float()
hpreact = color @ W1
hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
h = torch.tanh(hpreact)
logits = torch.tanh(h @ W2)
print(logits)
logits.softmax(dim=1)

tensor([[ 0.0092, -0.0400, -0.0079, -0.0507, -0.0257, -0.1004, -0.0676,  0.0631,
          0.0312, -0.0460,  0.0321]], grad_fn=<TanhBackward0>)


tensor([[0.0934, 0.0889, 0.0918, 0.0879, 0.0901, 0.0837, 0.0865, 0.0985, 0.0954,
         0.0883, 0.0955]], grad_fn=<SoftmaxBackward0>)

In [28]:
with torch.no_grad():
  # pass the training set through
  hpreact = inputs @ W1 # + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnstd = hpreact.std(0, keepdim=True)

In [34]:
color = torch.tensor([1,0,0]).float()
hpreact = color @ W1
hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
h = torch.tanh(hpreact)
logits = torch.tanh(h @ W2)
print(logits)
probs = logits.softmax(dim=1)
print(probs)
print(categories[torch.argmax(probs)])

tensor([[-0.9517, -1.0000,  1.0000, -0.9990, -0.9504,  0.9628,  0.8474,  0.0220,
          0.9986, -0.9632, -0.7376]], grad_fn=<TanhBackward0>)
tensor([[0.0280, 0.0267, 0.1973, 0.0267, 0.0281, 0.1901, 0.1694, 0.0742, 0.1970,
         0.0277, 0.0347]], grad_fn=<SoftmaxBackward0>)
Brown
